In [58]:
from selenium import webdriver
import requests
import time
from sanitize_filename import sanitize

In [2]:
driver = webdriver.Chrome()

In [3]:
link = 'https://www.linkedin.com/learning/revit-generative-design/applied-generative-design-for-architecture-and-engineering'

In [4]:
driver.get(link)

In [50]:
sidebar = driver.find_elements_by_class_name('classroom-toc-item')

In [62]:
def videoLinks(sidebar):
    collection = []
    for i, e in enumerate(sidebar):
        link = e.find_element_by_tag_name('a').get_attribute('href')
        name = e.find_element_by_class_name('classroom-toc-item__title').text
        if link.find('quiz') > 0:
            pass
        else:
            collection.append([str(i) + ' - ' + sanitize(name), link])
    return collection

In [63]:
links = videoLinks(sidebar)

In [69]:
def downloadAllVideos(driver, link):
    driver.get(link)
    sidebar = driver.find_elements_by_class_name('classroom-toc-item')
    pairs = videoLInks(sidebar)
    for p in pairs:
        time.sleep(5)
        downloadVideo(driver, p)

In [66]:
def downloadVideo(driver, pair):
    [name, link] = pair
    driver.get(link)
    time.sleep(5)
    videoFrame = driver.find_element_by_tag_name('video').get_attribute('src')
    s = requests.Session()
    response = s.get(videoFrame, stream=True)
    print(response.status_code)
    with open('ll/' + name +'.mp4', 'wb') as f:
        f.write(response.content)

In [67]:
downloadAllVideos(driver, links)

200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200
200


200
